# Estudiantes
## Luis Mariano Gutiérrez Divas - 1535719
## Adolfo Guillermo Turcios Brizuelas - 1546712

In [1]:
import cv2
import mediapipe as mp
import numpy as np
from math import acos, degrees

def palm_centroid(coordinates_list):
    coordinates = np.array(coordinates_list)
    centroid = np.mean(coordinates, axis=0)
    centroid = int(centroid[0]), int(centroid[1])
    return centroid

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

thumb_points = [1, 2, 4]
palm_points = [0, 1, 2, 5, 9, 13, 17]
finger_tips_points = [8, 12, 16, 20]
finger_base_points = [6, 10, 14, 18]

# Colores
WHITE = (255, 255, 255)
PURPLE = (81, 46, 95)
YELLOW = (0, 255, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)

RED = (0, 0, 255)
ORANGE = (0, 165, 255)
PINK = (203, 192, 255)
BLACK = (0, 0, 0)

with mp_hands.Hands(
    model_complexity=1,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)
        
        total_fingers_counter = 0
        thickness = [2, 2, 2, 2, 2]
        show_rock_roll = False
        show_paz = False
        show_demon = False
        show_ronal = False
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                coordinates_thumb = []
                coordinates_palm = []
                coordinates_ft = []
                coordinates_fb = []

                for index in thumb_points:
                    x = int(hand_landmarks.landmark[index].x * width)
                    y = int(hand_landmarks.landmark[index].y * height)
                    coordinates_thumb.append([x, y])
                
                for index in palm_points:
                    x = int(hand_landmarks.landmark[index].x * width)
                    y = int(hand_landmarks.landmark[index].y * height)
                    coordinates_palm.append([x, y])
                
                for index in finger_tips_points:
                    x = int(hand_landmarks.landmark[index].x * width)
                    y = int(hand_landmarks.landmark[index].y * height)
                    coordinates_ft.append([x, y])
                
                for index in finger_base_points:
                    x = int(hand_landmarks.landmark[index].x * width)
                    y = int(hand_landmarks.landmark[index].y * height)
                    coordinates_fb.append([x, y])
                
                # Pulgar
                p1 = np.array(coordinates_thumb[0])
                p2 = np.array(coordinates_thumb[1])
                p3 = np.array(coordinates_thumb[2])
                l1 = np.linalg.norm(p2 - p3)
                l2 = np.linalg.norm(p1 - p3)
                l3 = np.linalg.norm(p1 - p2)
                angle = degrees(acos((l1**2 + l3**2 - l2**2) / (2 * l1 * l3)))
                thumb_finger = angle > 150

                # Índice, medio, anular y meñique
                nx, ny = palm_centroid(coordinates_palm)
                coordinates_centroid = np.array([nx, ny])
                coordinates_ft = np.array(coordinates_ft)
                coordinates_fb = np.array(coordinates_fb)
                d_centroid_ft = np.linalg.norm(coordinates_centroid - coordinates_ft, axis=1)
                d_centroid_fb = np.linalg.norm(coordinates_centroid - coordinates_fb, axis=1)
                dif = d_centroid_ft - d_centroid_fb
                fingers = dif > 0
                fingers = np.append(thumb_finger, fingers)
                
                total_fingers_counter += np.count_nonzero(fingers == True)
                
                # Verificar si pulgar, índice y meñique están levantados
                if fingers[0] and fingers[1] and fingers[4] and not fingers[2] and not fingers[3]:
                    show_rock_roll = True

                if fingers[1] and fingers[2] and not fingers[0] and not fingers[3] and not fingers[4]:
                    show_paz = True

                if fingers[1] and fingers[4] and not fingers[0] and not fingers[2] and not fingers[3]:
                    show_demon = True

                if fingers[0] and fingers[4] and not fingers[1] and not fingers[2] and not fingers[3]:
                    show_ronal = True

                for i, finger in enumerate(fingers):
                    if finger:
                        thickness[i] = -1

                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )
        
        cv2.rectangle(frame, (5, 5), (55, 55), (0, 0, 0), -1)
        cv2.putText(frame, str(total_fingers_counter), (15, 50), 1, 3, (255, 255, 255), 2)
        cv2.rectangle(frame, (5, 90), (55, 140), WHITE, thickness[0])
        cv2.putText(frame, "Pulgar", (5, 160), 1, 1, WHITE, 2)
        cv2.rectangle(frame, (5, 170), (55, 220), PURPLE, thickness[1])
        cv2.putText(frame, "Indice", (5, 240), 1, 1, PURPLE, 2)
        cv2.rectangle(frame, (5, 250), (55, 300), YELLOW, thickness[2])
        cv2.putText(frame, "Medio", (5, 320), 1, 1, YELLOW, 2)
        cv2.rectangle(frame, (5, 330), (55, 380), GREEN, thickness[3])
        cv2.putText(frame, "Anular", (5, 400), 1, 1, GREEN, 2)
        cv2.rectangle(frame, (5, 410), (55, 460), BLUE, thickness[4])
        cv2.putText(frame, "Menique", (5, 475), 1, 1, BLUE, 2)

        if show_rock_roll:
            cv2.putText(frame, "ROCK AND ROLL", (125, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)

        if show_paz:
            cv2.putText(frame, "PAZ", (125, 150), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)

        if show_demon:
            cv2.putText(frame, "DEMONIO", (125, 200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)

        if show_ronal:
            cv2.putText(frame, "RONALDINHO", (125, 250), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)

        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

D:\UBSOS\anaconda\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
